We have 6 classes in this classification task:
    1. barren_land (73396)
    2. building (14922)
    3. grassland (50346)
    4. road (8191)
    5. trees (56808)
    6. water (120331)

As indicated above, they are hugely inbalanced

It's highly likely that a CNN with a classifier at the end will over predict water and under predict road
Since we don't know the natural distribution of these images, we will attempt to balance these images out

How:
    1. Augmentation
    2. Subsampling/oversampling
    3. ???
    

May not be the best practice will need to look up subsampling methods

In [1]:
count_by_cat = {'barren_land' :73396, 'building':14922, 'grassland':50346, 'road':8191, 'trees':56808, 'water':120331}

In [2]:
count_by_cat_perc = {k: v/sum(count_by_cat.values())*100 for k,v in count_by_cat.items()}

In [3]:
count_by_cat_perc

{'barren_land': 22.653505929122144,
 'building': 4.605640845200837,
 'grassland': 15.53917665141947,
 'road': 2.5281332370352536,
 'trees': 17.53365803070427,
 'water': 37.139885306518025}

In [4]:
sum(count_by_cat.values())

323994

Now let's try protyping some of this

In [5]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torchvision import models, transforms, utils
from PIL import Image
from tqdm import tqdm
from torch.optim import Adam
import math
import cv2

In [26]:
class vgg_preloaded(nn.Module):

	def __init__(self, num_class, use_cuda):
		super(vgg_preloaded, self).__init__()
		self.use_cuda = use_cuda
		self.num_class = num_class
		self.dtype = torch.cuda.FloatTensor if self.use_cuda else torch.FloatTensor
		model = models.inception_v3(pretrained=True)
		self.model = model.cuda() if self.use_cuda else model
		for param in self.model.features.parameters():
			param.require_grad = False
		num_features = self.model.classifier[6].in_features
		features = list(self.model.classifier.children())[:-1] # Remove last layer
		features.extend([nn.Linear(num_features, self.num_class)])
		self.model.classifier = nn.Sequential(*features)
		self.model.classifier.require_grad = True

	def forward(self, inp):
		return(self.model(inp))

In [6]:
image_dir = '../../data/images/'
barren_land_dir = image_dir + 'barren_land/'
barren_land = os.listdir(barren_land_dir)

In [7]:
image_3 = cv2.imread(barren_land_dir + barren_land[3])

In [8]:
image_3.shape

(28, 28, 3)

In [31]:
inceptionv3 = models.inception_v3(pretrained=True)

In [32]:
inceptionv3

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

We need to check if everything is in png format

In [9]:
os.path.isdir(os.path.join(image_dir, os.listdir(image_dir)[0]))

False

In [34]:
# creating individual paths
sub_paths = [os.path.join(image_dir, file_dir) for file_dir in os.listdir(image_dir)]
# check if they're folders
sub_paths = [path for path in sub_paths if os.path.isdir(path)]

In [35]:
sub_paths

['../../data/images/water',
 '../../data/images/trees',
 '../../data/images/road',
 '../../data/images/barren_land',
 '../../data/images/building',
 '../../data/images/grassland']

In [21]:
from tqdm import tqdm
res = [0 if file_name[-3:] == 'png' else 1 for file_name in tqdm(os.listdir(sub_paths[0]))]

100%|██████████| 120332/120332 [00:00<00:00, 1170304.41it/s]


In [25]:
res = {}
for path in sub_paths:
    res[path] = sum([0 if file_name[-3:] == 'png' else 1 for file_name in tqdm(os.listdir(path))])

100%|██████████| 50347/50347 [00:00<00:00, 1180304.41it/s]


In [26]:
res

{'../../data/images/barren_land': 0,
 '../../data/images/building': 0,
 '../../data/images/grassland': 0,
 '../../data/images/road': 0,
 '../../data/images/trees': 0,
 '../../data/images/water': 0}

OK! 

so all our images are of '.png' format

In [27]:
sub_paths

'../../data/images/'

In [73]:
class_dict = {'water':1, 'trees':2, 'road':3, 'barren_land': 4, 'building': 5, 'grassland':6}

all_files_dict = {}
for path in sub_paths:
    print(os.path.split(path)[1])
    all_files_dict = {**all_files_dict, **{os.path.join(path, file_name):class_dict[os.path.split(path)[1]] for file_name in  os.listdir(path)}}

water
trees
road
barren_land
building
grassland


In [66]:
len(all_files_dict)

324000

Ok officially we have 32.4k images

In [75]:
set(all_files_dict.values())

{1, 2, 3, 4, 5, 6}

In [76]:
import pandas as pd

In [77]:
test_df = pd.DataFrame.from_dict({'col1': ['a','b','c','d'], 'col2':[1,2,3,4]})

In [79]:
test_df.loc[test_df['col1'] == 'a', 'col2']

0    1
Name: col2, dtype: int64

In [86]:
import numpy as np
nparrayres = np.array(test_df.loc[test_df['col1'] == 'a', 'col2'])
nparrayres

array([1])

In [88]:
torch.from_numpy(nparrayres)[0]

tensor(1)